# Active inference

In this notebook we will go through the main ideas and concepts described in this book: https://mitpress.mit.edu/9780262045353/active-inference/

The examples we will be revieing and implementing is well described here: https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5167251/

### Note

In case you are using `venv`, you need to launch the following command before being able to run the jupyter notebook with the appropriate kernel:

```
python -m ipykernel install --user --name=<virtualenv_name>
```

Then, inside jupyter lab, check the top right to use the kernel named `<virtualenv_name>`.

In [1]:
# convenient scripts
%reload_ext autoreload
%autoreload 2

# imports
import numpy as np
import plotly.express as px
from itertools import product
import math



## Mathematical modelling

### Biological principles

Sentient beings act on the world with one goal in mind: **survival**. Survival is maximised if **surprise is minimised**. What is surprise? It it the $log(1/P(y))$, where $y$ is a sensory input.

How can $P$ be estiamted? The sentient being has to have a **model of the world** where its beliefs (modelled as priors) fit otgether to provide a posterior $P$ given the data observed.

Furthermore, the way a senntient being act on the world and try to make sense of sensory inputs has also to have one goal inmind: minimise the prediction error of the model. This is equivalent to minimising the surprise.

The last statement can be formulated in the language of bayesian inference: hence the name "bayesian brain".

### Bayesian brain

The brain is an inference machine according to the bayesian brain hypothesis. Given the sensory inputs $y$ and the hidden parameters $x$ that are used to make sense of the world, the goal of the brain is to compute $P(y|x)$, a.k.a. the posterior.

Assuming that $P$ is the actual distribution of the world responses, the sentient being needs to find a good approximation of this distribution: this approximation will be denoited $Q$. How can we compute $Q$? By minimising free energy (see below).

### Markov blanket

Given three random variables $X, Y, B$, then $B$ is a blanket for $X,Y$ iff $X$ and $Y$ are conditionally independent w.r.t. $B$, i.e. $p(X, Y | B) = p(X | B) p(Y | B)$.

This concept is essential to be able to define the "self", i.e. the system as separated from the outer world. In particular, the action and sensory inputs are the markov blanket for the hidden states of the world and the self.

## Free energy

How can we compute the approximateposterior $Q$? Via a free energy minimisation.

### Variational free energy

We would like to write a funcitonal of $Q$ such that, by minimising it w.r.t. $Q$, then $Q$ is the closest possible to the actual distribution $P$. This will also depend on the sensory input $y$.

The minimisation of the free energy will then give us the best $Q$ possible, i.e. the one closest to $P$. However, let's recall that the sentient being is not just about finding $Q$ (this would correspond to the epistemic gain, i.e. optimising your knowledge of the world), but it also has to deal with making actions that keep the sistem in a controlled state! This is the aspect about minimising the surpise.

Hence the free energy is:

$$F(Q,y) = -\mathbb E_{Q(x)}(log P(x,y)) - H(Q(x)),$$

where the entropy $H$ is 

$$ H(Q(x)) = \mathbb E_{Q(x)}(log(Q(x))).$$

Recalling that $P(x,y)=P(x|y)P(y)$ by the Bayes theorem, one can manipulate the above to

$$F(Q,y) = D_{KL}(P(x|y)||Q(x))-log(P(y)),$$

where the $D_{KL}$ is the error between the internal world model and the actual world process (this is about optimising one's beliefs), and the last term $-log(P(y))=log(1/P(y))$ is the surprise (that is optimised via actions).

### Expected free energy

When **planning** needs to be involved, how can we model it? How can we learn it? Again, minimising the "expected" free energy. This is a crucial point: we need to leverage our world model to make prediction of the future and evaluate each policy. Even with coutnerfactuals! If our model is good, then we will get a good policy $\pi$ that will drive the being behaviour. A policy is a sequence of actions.

The energy, being about the future, need to be averaged over all possible outcomes given a policy $\pi$. Then, we can minimise the energy to obtain the best $\pi$.

The expected free energy is (notice that now both $x$ and $y$ are integrated):

$$G(\pi) = -\mathbb E_{Q(x,y|\pi)}(log P(x,y|C)) - H(Q(x|\pi)) \ge \mathbb -E_{Q(x,t | \pi)}(D_{KL}(Q(x |y, \pi) || Q(x | \pi))) + \mathbb E_{Q(y | \pi)}(log P(y|C))$$

where $Q(x,y|\pi) := Q(x | \pi)P(y | x)$ and $C$ are parameters (we will use it to describe the tendency to move towards food and go away from poison). The original definition uses the second expression, not the first one as I did, to define $G(\pi)$: however, my preference, in analogy with the Variational free energy, is to slightly change the convention.

Note that to find $Q$, we need to rely on the minimisation of $F$ and add a conditioning on the policy $\pi$. So, with a given and fixed $\pi$:

$$ F_\pi(Q,y) = -\mathbb E_{Q(x|\pi)}(log P(x,y|\pi)) - H(Q(x|\pi)) $$

Notation: in the above equations, $x$ and $y$ are sequences of hidden states and sensory data respectively, of the same time length of the policy!

## Learning

**Learning is about doing inference** on the parameters of the distributions involved in the game.

This means that the formalism stays the same, except that now some of the distributions will have parameters of which we have a prior belief. Thus, when minimising, we want to also improve those parameters. Usually, in emprical bayes, this is done via the upgrade of the prior distribution $p(\theta)$ with the posterior $p(\theta|y)$, where $\theta$ is a parameter and $y$ the data.

You will see, in the example below, that we will be building the world process $P$ using some pre-defined matrices $A$, $B$. Adding priors associated to these matrices in the approximate posterior $Q$, i.e. having $Q$ being a distribution over the space of matrices $A$, $B$... as well, is the first step towards learning. The subsequent step would be to minimise the free energy to find $Q$, including the part over the matrix sapces. We will not add these priors in our computations.

## How about attention?

Attention is already embedded in this theory, as the way knowledge is chosen to validate our world model world model. Said differently, attention is the process of selecting the highest quality data from what we have already mea­sured and using ­these to validate our world model preditions. The
design of the next experiment to ensure the highest quality data is referred to as **saliency**.

## Example

Let's now build everything in the discrete setting.

Our world model will be a POMDP, described by the following bayesian network:

![pomdp](pomdp.jpg)

This network corresponds to this probability distribution:

$$ P(x,y,\pi,C) = P(C)P(x_0)P(\pi)\prod_t^T P(y_t|x_t)P(x_{t+1}|x_t, \pi_t)$$

Also in this case, $x,y$ are sequences of states, depending on $t$!
We then have to specify what are the spaces in which the hidden states, the observations etc... belong to.

So, let's consider a small agent in a 1D world. The world has **4 spacial sites**, in each of them there can be nothing, food or poison. So $y_t \in \{0,1,2\}$, where `0` is poison, `1` is nothing, `2` is food.

The hidden state $x_t$ at time $t$ models the position in the 1D world and is assumed to be a number in $\{0,1,2,3\}$.

![world](world.jpg)

The possible **actions** of the agent are to move right or left . In particular `right = 0, left = 1`.



In [2]:

print("policy")
pi = [1,1,1,1]  # this is a sequence of actions
number_of_policies = 2**4
max_time = len(pi)
number_of_locations = 4
print("Q(x|pi)")  # x = (s0,s1,s2,...,sT)
Q = np.random.random((max_time, number_of_locations, number_of_policies))  # we have to find it by minimisation! The dimensions are: 
# - time (which has dim = 4),
# - hidden state space (which has dim = number_of_locations), 
# - policy (which has dim = number_of_policies)


# normalise Q:
def normaliseQ():
    for t in range(max_time):
        for j in range(number_of_policies):
            norm = np.sum([Q[t, i, j] for i in range(number_of_locations)])
            for i in range(number_of_locations):
                if norm == 0:
                    raise ValueError(f"The modelling must be flawed in row {i}")
                Q[t, i, j] = Q[t, i, j]/norm
        for i in range(number_of_locations):
            for j in range(number_of_policies):
                if Q[t, i, j]<=0:
                    Q[t, i, j] = 0.0001

normaliseQ()  # in place

print("P(y_t|x_t)")  # action at each site
A = np.array([[0.9,0,0,0.1],[0,1,1,0],[0.1,0,0,0.9]])  # dimensions 3 x number_of_locations

print("P(x_0)")  # initial position x_0=1
D = np.array([0,1,0,0])  # dimensions number_of_locations

print("P(C)")  # this is the preference for food and dislike of poison
C = np.array([0.01,0.1,0.1,0.79])  # dimensions number_of_locations

print("P(x_{t+1}|x_t, \pi_t)")  # how to move
B = np.zeros([number_of_locations, number_of_locations, 3])  # dimensions: number_of_locations x number_of_locations x 3
# eating
B[0,0,2] = 1
B[1,1,2] = 1
B[2,2,2] = 1
B[3,3,2] = 1

# move right
B[1,0,0] = 1
B[2,1,0] = 1
B[3,2,0] = 1
B[3,3,0] = 1

# move left
B[0,1,1] = 1
B[1,2,1] = 1
B[2,3,1] = 1
B[0,0,1] = 1

# normalisation as it is a probability:
for j in range(number_of_locations):
    for k in range(3):
        norm = np.sum([B[i,j,k] for i in range(number_of_locations)])
        for i in range(number_of_locations):
            if norm == 0:
                raise ValueError(f"The modelling must be flawed in row {i}")
            B[i,j,k] = B[i,j,k]/norm


print("G(\pi)")
def G(pi):
    """ expected free energy
    pi: policy, a list
    """
    k = np.sum([pi[i]*2**i for i in range(max_time)])
    return np.sum([np.sum([-np.log(0.0000001+np.prod([A[j,i]*(D[i] if t == 0 else 1/number_of_locations)*C[i] for t in range(max_time)]))*np.prod([Q[t,i,k]*A[j,i] for t in range(max_time)]) for j in range(3)]) + np.prod([Q[t,i, k] for t in range(max_time)])*np.log(0.0000001+np.prod([Q[t,i, k] for t in range(max_time)])) for i in range(number_of_locations)])  # y -> j, x -> i, pi -> k, time -> t

G(pi)
print("F_\pi")

def y_from_pi(pi):
    """ get the list of y, given pi
    pi: policy, a list
    """
    pos = [1,1,1,1]
    for t in range(max_time):
        if pi[t] == 0:
            for j in range(t,max_time):
                pos[j] += 1
                pos[j] = min(pos[j],3)
        else:
            for j in range(t,max_time):
                pos[j] -= 1
                pos[j] = max(pos[j],0)
    #print(pos)
    return [0 if pos[t]<=0 else (2 if pos[t] >=3 else 1) for t in range(max_time)]


assert y_from_pi([0,0,0,0]) == [1,2,2,2]

def F(pi):
    """ variational free energy
    pi: policy, a list
    y: the list of sensory inputs
    """
    k = np.sum([pi[i]*2**i for i in range(max_time)])
    j = y_from_pi(pi)
    return np.sum([-np.log(0.0000001+np.prod([A[j[t],i]*B[i,j[t],pi[t]] for t in range(max_time)]))*np.prod([Q[t, i, k] for t in range(max_time)]) + np.prod([Q[t,i, k] for t in range(max_time)])*np.log(0.0000001+np.prod([Q[t,i, k] for t in range(max_time)])) for i in range(number_of_locations)])
F(pi)
print("All initialised")

policy
Q(x|pi)
P(y_t|x_t)
P(x_0)
P(C)
P(x_{t+1}|x_t, \pi_t)
G(\pi)
F_\pi
All initialised


## Optimisation

The next block of code is about finding the optimum $Q$ and optimum $pi$.

To do so, we will proceed iteratively, a bit like in the expectation-maximisation problems:
 - first, given the policy $\pi$, we minimise $F$ to get the novel $Q$
 - secondly, we will minimise $G$ to find the optimum policies, and since there are only $3^4$ cases, the optimum will be global

The two steps above are iterated a few times, till they converge.

### Differentiability

We could actually differentiate the variational free energy and get the explicit form of $\nabla_Q F_\pi(Q,y)$. Then, once can proceed with the gradient descent method to find the optimal $Q$. We are leaving this step for the interested reader.

In [3]:
all_possible_pi = [ele for ele in product(range(2), repeat = 4)]
assert len(all_possible_pi) == number_of_policies


In [4]:
Fs = []
MAX_ITER = 10
pi_old = pi
for _ in range(MAX_ITER):
    
    # step 1
    val_old = F(pi)
    for a in range(1000):
        Q_old = Q.copy()
        Q = Q + 0.01*(np.random.random((max_time, number_of_locations, number_of_policies)) - 0.5)
        normaliseQ()
        val_new = F(pi)
        Fs.append(val_old)
        if val_new < val_old:
            val_old = val_new
        else:
            Q = Q_old

    # step 2
    pos = np.argmin(list(map(G, all_possible_pi)))
    pi = all_possible_pi[pos]
    #if pi == pi_old:
    #    print("Done!")
    #    break
    pi_old = pi
    print(pi)

fig = px.scatter(Fs)
fig.show(renderer="iframe")


(0, 0, 0, 0)
(0, 0, 0, 0)
(0, 0, 0, 0)
(0, 0, 0, 0)
(0, 0, 0, 0)
(0, 0, 0, 0)
(0, 0, 0, 0)
(0, 0, 0, 0)
(0, 0, 0, 0)
(0, 0, 0, 0)


In [5]:

fig = px.imshow(Q[0], labels=dict(x="policy", y="x", color="Probability"),
                x=list(map(str, all_possible_pi)),
                y=['0', '1', '2', '3'])
fig.show(renderer="iframe")

## Conclusion

One expects that $Q$ approximates $P$ and that the chosen policy is an effective one in finding food and moving towards it. So, the optimum policy shall shave more `0`s than `1`s.